In [1]:
%time import requests
%time from pymongo import MongoClient
%time import ast
%time import pandas as pd
%time import numpy as np


CPU times: user 38.7 ms, sys: 33.3 ms, total: 72 ms
Wall time: 651 ms
CPU times: user 28.2 ms, sys: 26.9 ms, total: 55.1 ms
Wall time: 764 ms
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs
CPU times: user 529 ms, sys: 186 ms, total: 716 ms
Wall time: 6.66 s
CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.06 µs


In [2]:
%time client = MongoClient()
%time db = client.meetup
groups_col=db.groups
%time groups_clean_extra_col = db.groups_clean_extra
groups_clean_extra_col
test_col = db.test

CPU times: user 1.5 ms, sys: 1.45 ms, total: 2.94 ms
Wall time: 2.35 ms
CPU times: user 58 µs, sys: 52 µs, total: 110 µs
Wall time: 87 µs
CPU times: user 48 µs, sys: 1 µs, total: 49 µs
Wall time: 56 µs


In [ ]:
api_key='466c256e2d1d455e4d3e4861bd32b'

In [ ]:
'https://api.meetup.com/2/groups?zip=94109&offset=0&city=San+Francisco&format=json&lon=-122.419998169&photo-host=public&state=ca&page=20&radius=25.0&fields=sponsors%2Clast_event%2Cnext_event%2Cphotos%2C+short_link&lat=37.7900009155&order=id'

In [ ]:
def save_groups_raw(api_key,location,radius,offset):
    groups = requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location='+location+\
                          '&radius='+str(radius)+'&page=200&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text)
    for group in groups_info:
        group['cleaned']=False
        group['member_profiles']=[]
        group['description']=clean_description(group['description'])
        inserted_group = groups_col.insert_one(group)
        print inserted_group.inserted_id

        

In [ ]:
def save_groups_clean(api_key,location,radius,offset):
    groups = requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location='+location+\
                          '&radius='+str(radius)+'&page=200&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text)
    for group in groups_info:
        group['cleaned']=True
        group['member_profiles']=[]
        group['events']=[]
        try:
            group['next_event_id']=group['next_event']['id']
            group['next_event_name']=group['next_event']['name']
            group['next_event_time']=group['next_event']['time']
            group['next_event_offset']=group['next_event']['utc_offset']
            group['next_event_rsvp']=group['next_event']['yes_rsvp_count']
            group['organizer_id'] = group['organizer']['id']
            group['organizer_name'] = group['organizer']['name']
        except KeyError:
            group['next_event_id']=np.nan
            group['next_event_name']=np.nan
            group['next_event_time']=np.nan
            group['next_event_offset']=np.nan
            group['next_event_rsvp']=np.nan
            group['organizer_id'] = np.nan
            group['organizer_name'] = np.nan
        group['description']=clean_description(group['description'])
        inserted_group = groups_clean_col.insert_one(group)
        print inserted_group.inserted_id

In [ ]:
def test(api_key,location,radius,offset,zip_code):
    groups = requests.get('https://api.meetup.com/2/groups?key='+api_key+'&sign=true&photo-host=public&zip='+str(zip_code)+'&fields=sponsors,last_event,next_event,photos, short_link&city='+location+'&state=CA&page=20&offset='+str(offset))
    return groups

groups_test=test(api_key,'San Francisco',20,0,94109)

In [ ]:
group_lst=ast.literal_eval(groups_test.text.replace('\\/','/'))
group_lst['results'][10]['description']


In [ ]:
def save_groups_clean_extra_info(api_key,location,radius,offset,zip_code,state):
    groups = requests.get('https://api.meetup.com/2/groups?key='+api_key+'&sign=true&photo-host=public&zip='+str(zip_code)+'&fields=sponsors,last_event,next_event,photos, short_link,description&city='+location+'&state='+state+'&page=100&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text.replace('\\/','/'))
    for group in groups_info['results']:
        #print group
        group['cleaned']=True
        group['member_profiles']=[]
        group['events']=[]
        #print group['description']
        try:
            group['next_event_id']=group['next_event']['id']
            group['next_event_name']=group['next_event']['name']
            group['next_event_time']=group['next_event']['time']
            group['next_event_offset']=group['next_event']['utc_offset']
            group['next_event_rsvp']=group['next_event']['yes_rsvp_count']
            group['organizer_id'] = group['organizer']['id']
            group['organizer_name'] = group['organizer']['name']
            #group['description']=clean_description(group['description'])
        except KeyError:
            group['next_event_id']=np.nan
            group['next_event_name']=np.nan
            group['next_event_time']=np.nan
            group['next_event_offset']=np.nan
            group['next_event_rsvp']=np.nan
            group['organizer_id'] = np.nan
            group['organizer_name'] = np.nan
            #group['description']=np.nan
        
        inserted_group = groups_clean_extra_col.insert_one(group)
        #print inserted_group.inserted_id

In [ ]:
for offset in range(88):
    save_groups_clean_extra_info(api_key,'san francisco',30,offset,94109,'CA')
    print offset

In [ ]:
#for offset in range(36,44):
for offset in range(36,44):
    save_groups(api_key,'san francisco',30,offset)
    print offset

In [ ]:
groups=requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location=san francisco&radius=30&page=200&offset=1')

In [ ]:
groups

In [ ]:
groups_info = ast.literal_eval(groups.text)


In [ ]:
len(groups_info)

In [ ]:
for group in groups_info:
    inserted_group = groups_col.insert_one(group)
    print inserted_group.inserted_id

In [5]:
%time tech_groups=groups_clean_extra_col.find({})

CPU times: user 147 µs, sys: 24 µs, total: 171 µs
Wall time: 180 µs


In [ ]:
counter=0
for i in tech_groups:
    counter+=1
print counter

In [6]:
%time tech_groups_arr=[i for i in tech_groups]


CPU times: user 5.56 s, sys: 411 ms, total: 5.97 s
Wall time: 6.13 s


In [ ]:
tech_groups_arr[:10]

In [8]:
%time groups_df = pd.DataFrame(tech_groups_arr)
groups_df.shape

CPU times: user 192 ms, sys: 11.4 ms, total: 203 ms
Wall time: 209 ms


(7951, 37)

In [ ]:
groups_df.columns

In [ ]:
#sanfran_tech=groups_df[groups_df.category=={u'shortname': u'Tech', u'id': 34, u'name': u'Tech', u'sort_name': u'Tech'}]
groups_df.category.describe()

In [7]:
def get_category_name(category):
    try:
        return category['name']
    except TypeError:
        return np.nan
    
def get_category_id(category):
    try:
        return category['id']
    except TypeError:
        return np.nan
        


In [9]:
%time groups_df['category_name'] = groups_df.category.map(get_category_name)
%time groups_df['category_id']=groups_df.category.map(get_category_id)


CPU times: user 10.9 ms, sys: 2.01 ms, total: 12.9 ms
Wall time: 12.2 ms
CPU times: user 11.3 ms, sys: 1.89 ms, total: 13.2 ms
Wall time: 11.7 ms


In [ ]:
groups_df.info()

In [11]:
def get_next_event_id(event):
    try:
        return event['id']
    except TypeError:
        return np.nan

def get_next_event_name(event):
    try:
        return event['name']
    except TypeError:
        return np.nan

def get_next_event_time(event):
    try:
        return event['time']
    except TypeError:
        return np.nan
    
def get_next_event_offset(event):
    try:
        return event['utc_offset']
    except TypeError:
        return np.nan
    
def get_next_event_rsvp(event):
    try:
        return event['yes_rsvp_count']
    except TypeError:
        return np.nan 

def get_organizer_id(organizer):
    try:
        return organizer['id']
    except TypeError:
        return np.nan

def get_organizer_name(organizer):
    try:
        return organizer['name']
    except TypeError:
        return np.nan


In [12]:
%time groups_df['next_event_id']=map(get_next_event_id,groups_df.next_event)
%time groups_df['next_event_name']=map(get_next_event_name,groups_df.next_event)
%time groups_df['next_event_time']=map(get_next_event_time,groups_df.next_event)
%time groups_df['next_event_offset']=map(get_next_event_offset,groups_df.next_event)
%time  groups_df['next_event_rsvp']=map(get_next_event_rsvp,groups_df.next_event)
%time groups_df['organizer_id']=map(get_organizer_id,groups_df.organizer)
%time  groups_df['organizer_name']=map(get_organizer_name,groups_df.organizer)

CPU times: user 25.8 ms, sys: 3.28 ms, total: 29.1 ms
Wall time: 27 ms
CPU times: user 27.1 ms, sys: 3.48 ms, total: 30.5 ms
Wall time: 31.9 ms
CPU times: user 24.4 ms, sys: 2.06 ms, total: 26.5 ms
Wall time: 26.9 ms
CPU times: user 27.5 ms, sys: 868 µs, total: 28.4 ms
Wall time: 28.9 ms
CPU times: user 27.6 ms, sys: 1.32 ms, total: 28.9 ms
Wall time: 45.5 ms


KeyError: 'id'

In [ ]:
groups_df.description[4]

In [13]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_backspaces(text):
    split_text = text.split()
    for i in split_text:
        i.replace('\n',' ')
    return ' '.join(split_text)

def clean_description(descr):
    no_tags = strip_tags(groups_df.description[199])
    return remove_backspaces(no_tags)

In [15]:
%time groups_df.description = groups_df.description.apply(clean_description)

CPU times: user 678 ms, sys: 22.5 ms, total: 700 ms
Wall time: 846 ms


In [ ]:
groups_df.description[10]